In [1]:
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 2.5 MB/s eta 0:00:00a 0:00:01


In [2]:
from kafka import KafkaProducer
import csv
import json

producer = KafkaProducer(bootstrap_servers="localhost:9092", value_serializer=lambda v: json.dumps(v).encode('utf-8'))
i = 0
with open("./dataset.csv") as f:
    reader = csv.DictReader(f)
    for row in reader:
        i = i + 1
        producer.send(topic="mytopic", value=row)
        producer.flush()
        if i == 10:
            break

In [3]:
from kafka import KafkaConsumer

consumer = KafkaConsumer("mytopic", bootstrap_servers="localhost:9092", auto_offset_reset='earliest', consumer_timeout_ms=10000)
data = []
for message in consumer:
    data.append(json.loads(message.value))
consumer.close()

with open("output_dataset.json", "w+") as final:
    json.dump(data, final)

In [17]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ProiectIBD').config("spark.jars.packages", "mysql:mysql-connector-java:8.0.33").getOrCreate()

In [5]:
df = spark.read.json("output_dataset.json")
df.show(5)

+---------+-----------+-----------+----------+---------------+-----------------+-----+----------+----------+------------------+--------------------+-------------------+---------------+-----+--------------------+--------------+--------+----------+----------+
|AGE_GROUP|ARREST_BORO|ARREST_DATE|ARREST_KEY|ARREST_PRECINCT|JURISDICTION_CODE|KY_CD|LAW_CAT_CD|  LAW_CODE|          Latitude|             Lon_Lat|          Longitude|      OFNS_DESC|PD_CD|             PD_DESC|     PERP_RACE|PERP_SEX|X_COORD_CD|Y_COORD_CD|
+---------+-----------+-----------+----------+---------------+-----------------+-----+----------+----------+------------------+--------------------+-------------------+---------------+-----+--------------------+--------------+--------+----------+----------+
|    45-64|          M| 11/22/2021| 236791704|             28|                0|     |         M|PL 2225001|40.799008797000056|POINT (-73.952408...| -73.95240854099995|               |  581|                    |         BLACK|

In [7]:
df.printSchema()

root
 |-- AGE_GROUP: string (nullable = true)
 |-- ARREST_BORO: string (nullable = true)
 |-- ARREST_DATE: string (nullable = true)
 |-- ARREST_KEY: string (nullable = true)
 |-- ARREST_PRECINCT: string (nullable = true)
 |-- JURISDICTION_CODE: string (nullable = true)
 |-- KY_CD: string (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- LAW_CODE: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Lon_Lat: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- PD_CD: string (nullable = true)
 |-- PD_DESC: string (nullable = true)
 |-- PERP_RACE: string (nullable = true)
 |-- PERP_SEX: string (nullable = true)
 |-- X_COORD_CD: string (nullable = true)
 |-- Y_COORD_CD: string (nullable = true)



In [12]:
import pyspark.sql.functions as F

df_pd = df.select(F.monotonically_increasing_id().alias("pd_id"), F.col("PD_CD"), F.col("PD_DESC")).distinct()
df_pd.show()

+-----+-----+--------------------+
|pd_id|PD_CD|             PD_DESC|
+-----+-----+--------------------+
|    3|  511|CONTROLLED SUBSTA...|
|    4|  177|        SEXUAL ABUSE|
|    6|  579|                    |
|    1|  153|              RAPE 3|
|    9|  153|              RAPE 3|
|    8|  729|FORGERY,ETC.,UNCL...|
|    2|  681|CHILD, ENDANGERIN...|
|    7|  263|         ARSON 2,3,4|
|    5|     |                    |
|    0|  581|                    |
+-----+-----+--------------------+



In [15]:
df_arrests = df.select(F.col("ARREST_DATE"), F.col("JURISDICTION_CODE"), F.col("PD_CD").alias("temp_pd_cd"), F.col("PD_DESC").alias("temp_pd_desc"))
df_arrests = df_arrests.join(df_pd, (df_arrests["temp_pd_cd"] == df_pd["PD_CD"]) & (df_arrests["temp_pd_desc"] == df_pd["PD_DESC"])).select("ARREST_DATE", "JURISDICTION_CODE", "pd_id")
df_arrests.show()

+-----------+-----------------+-----+
|ARREST_DATE|JURISDICTION_CODE|pd_id|
+-----------+-----------------+-----+
| 11/22/2021|                0|    0|
| 12/04/2021|                0|    9|
| 12/04/2021|                0|    1|
| 11/09/2021|                0|    2|
| 06/18/2007|                1|    3|
| 01/26/2019|                0|    4|
| 02/06/2019|                0|    5|
| 12/03/2021|                0|    6|
| 11/10/2021|               72|    7|
| 12/28/2021|                0|    8|
| 01/06/2016|                0|    9|
| 01/06/2016|                0|    1|
+-----------+-----------------+-----+



In [16]:
df_pd.write.format("jdbc").option("driver","com.mysql.cj.jdbc.Driver").option("url", "jdbc:mysql://localhost:3306/IBD").option("dbtable", "PD").option("user", "root").option("password", "123").mode("overwrite").save()

Py4JJavaError: An error occurred while calling o121.save.
: java.lang.ClassNotFoundException: com.mysql.cj.jdbc.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:592)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:246)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:250)
	at